In [1]:
# Подавление предупреждений
import warnings
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

import os
import glob
import re
import unicodedata
import json
import qbstyles
import pandas as pd
import bs4
import tqdm as tq
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import jupyterlab as jlab

from pathlib import Path
from bs4 import BeautifulSoup
from tqdm import tqdm
from qbstyles import mpl_style

from typing import List, Dict, Optional

mpl_style(dark = False)

In [2]:
# Определение пути к папке с CSV файлами
path_to_files = Path("/Users/dl/Library/Mobile Documents/com~apple~CloudDocs/Работа/ВШЭ/RecSys_24-25/Вакансии/v2/Обработанные")

# Получение списка всех CSV файлов
csv_files = list(path_to_files.rglob("*.csv"))

df = pd.DataFrame()

for file_path in tqdm(csv_files, desc = "Обработка CSV файлов", unit = "файл"):
    df_temp = pd.read_csv(file_path, sep = ';')
    df = pd.concat([df, df_temp], ignore_index = True)

Обработка CSV файлов: 100%|████████████████████████████████████████████████████████████████████| 117/117 [00:17<00:00,  6.83файл/s]


In [2]:
# Определение пути к папке с CSV файлами
path_to_files = Path("/Users/dl/Library/Mobile Documents/com~apple~CloudDocs/Работа/ВШЭ/RecSys_24-25/Связка_вакансии_дисциплины")
path_to_files = Path("/Users/dl/Downloads/results-2")

# Получение списка всех CSV файлов
csv_files = list(path_to_files.rglob("*.csv"))

df_u = pd.DataFrame()

for file_path in tqdm(csv_files, desc = "Обработка CSV файлов", unit = "файл"):
    df_temp = pd.read_csv(file_path, sep = ';')
    df_u = pd.concat([df_u, df_temp], ignore_index = True)

Обработка CSV файлов: 100%|███████████████████████████████████████████████████████████████████| 117/117 [00:00<00:00, 167.76файл/s]


In [4]:
len(df)

282866

In [3]:
len(df_u)

2759

In [6]:
df_u

,ID,Name,Description,KeySkills,ProfessionalRoles,City,SBERT_plus_LLM_Recommendations,SBERT_Disciplines
0,103566666,Руководитель группы по организации производства,Обязанности: Выполнять устные и письменные пр...,NaN,Руководитель отдела логистики,Тамбов,Лидерство и построение команды в бизнесе; Упра...,"Лидерство, коммуникации и персональная эффекти..."
1,104995340,Руководитель автотранспортной службы,Приглашаем кандидатов на вакансию Руководитель...,NaN,Руководитель отдела логистики,Фрязино (Московская область),Руководитель отдела логистики,Управление продажами; Современный менеджер: вв...
2,105040186,Менеджер по работе с клиентами в СДЭК,Ищем дополнительного сотрудника в связи с расш...,"Грамотная речь, Пользователь ПК, Прием и распр...",Руководитель отдела логистики,Москва,Аналитика цепей поставок; Бренд-менеджмент в ц...,Современный менеджер: введение в профессию; Ме...
3,104788897,Руководитель службы складского учета,Обязанности: - Организация работы складов на У...,"Excel, Документооборот, Работа с базами данных...",Руководитель отдела логистики,Красноярск,Управление складированием в цепях поставок; Уп...,Управление складированием в цепях поставок; Уп...
4,105372108,Руководитель отдела закупок и логистики,"Приветствую, наш будущий коллега! Меня зовут А...","Логистика поставок, Закупка мебели, Работа с п...",Руководитель отдела логистики,Москва,Управление цепями поставок в инновационных ком...,Менеджер продукта; Современный менеджер: введе...
...,...,...,...,...,...,...,...,...
2754,101393251,Начальник отдела стратегии и инноваций,Обязанности: Поиск и идентификация трендов р...,"Деловая переписка, Английский язык, Деловое об...",Менеджер/консультант по стратегии,Нижний Новгород,Маркетинговые стратегии; Управление продуктом ...,Продакт-менеджмент; Маркетинговые стратегии; У...
2755,105225915,Brand Strategist (middle+ / senior),Привет! Мы Signal (part of ONY) — агентство ис...,NaN,Менеджер/консультант по стратегии,Москва,Разработка бренд и коммуникационных стратегий;...,Разработка бренд и коммуникационных стратегий;...
2756,101625312,E-commerce strategist,Обязанности: Разработка еком-стратегий; Выбор...,NaN,Менеджер/консультант по стратегии,Москва,Маркетинговая стратегия в digital среде; Торго...,Навыки электронной коммерции; Разработка марке...
2757,105959727,"Бизнес аналитик в компанию ONE STORY, отдел ст...",ONE STORY − одна из крупнейших консалтинговых ...,"Работа в команде, MS PowerPoint, Проведение оп...",Менеджер/консультант по стратегии,Самара,Развитие клиентов и управление клиентским опыт...,Управление продуктом и продуктовая аналитика; ...


In [9]:
# Шаг 2: Определение общего набора столбцов для сопоставления (все кроме 'ID')
common_columns = list(set(df.columns) & set(df_u.columns))

common_columns

['Description', 'Name', 'KeySkills', 'City', 'ProfessionalRoles']

In [10]:
merged_df = pd.merge(df_u, df[['ID'] + common_columns], on=common_columns, how='left')

In [11]:
len(merged_df)

1187

In [13]:
merged_df

,Name,Description,KeySkills,ProfessionalRoles,City,SBERT_plus_LLM_Recommendations,SBERT_Disciplines,ID
0,Руководитель группы по организации производства,Обязанности: Выполнять устные и письменные пр...,NaN,Руководитель отдела логистики,Тамбов,Лидерство и построение команды в бизнесе; Упра...,"Лидерство, коммуникации и персональная эффекти...",103566666
1,Руководитель автотранспортной службы,Приглашаем кандидатов на вакансию Руководитель...,NaN,Руководитель отдела логистики,Фрязино (Московская область),Руководитель отдела логистики,Управление продажами; Современный менеджер: вв...,104995340
2,Менеджер по работе с клиентами в СДЭК,Ищем дополнительного сотрудника в связи с расш...,"Грамотная речь, Пользователь ПК, Прием и распр...",Руководитель отдела логистики,Москва,Аналитика цепей поставок; Бренд-менеджмент в ц...,Современный менеджер: введение в профессию; Ме...,105040186
3,Руководитель службы складского учета,Обязанности: - Организация работы складов на У...,"Excel, Документооборот, Работа с базами данных...",Руководитель отдела логистики,Красноярск,Управление складированием в цепях поставок; Уп...,Управление складированием в цепях поставок; Уп...,104788897
4,Руководитель отдела закупок и логистики,"Приветствую, наш будущий коллега! Меня зовут А...","Логистика поставок, Закупка мебели, Работа с п...",Руководитель отдела логистики,Москва,Управление цепями поставок в инновационных ком...,Менеджер продукта; Современный менеджер: введе...,105372108
...,...,...,...,...,...,...,...,...
1182,Директор по развитию бизнеса,Академия квантовой психологии и бизнеса пригла...,NaN,Менеджер/консультант по стратегии,Москва,Управленческое консультирование; Бизнес-психол...,Управленческое консультирование; Бизнес-психол...,104568848
1183,Ведущий консультант,В ОГКУ ЦЗН ТО по г.Тобольску и Тобольскому рай...,NaN,Менеджер/консультант по стратегии,Тобольск,Управление кадровым потенциалом; Управление че...,Карьерное консультирование персонала; Консалти...,103259421
1184,Chief of Staff,Our core values are: ✔️Excellence in everyth...,"Business Strategy, Business communication, Pro...",Менеджер/консультант по стратегии,Москва,Business Strategy; Business communication; Pro...,Управление человеческими ресурсами; Лидерство ...,105165780
1185,Антикризисный менеджер,В центральный офис федеральной компании рестор...,"Антикризисное управление, Стратегический менед...",Менеджер/консультант по стратегии,Уфа,Проектирование организационных структур управл...,Антикризисное управление; Управление продуктам...,104955465


In [20]:
import pandas as pd
from pathlib import Path
import shutil
from tqdm import tqdm

# Определение пути к папке с CSV файлами вакансий
path_to_files = Path("/Users/dl/Library/Mobile Documents/com~apple~CloudDocs/Работа/ВШЭ/RecSys_24-25/Вакансии/v2/Обработанные")

# Получение списка всех CSV файлов
csv_files = list(path_to_files.rglob("*.csv"))

# Определение пути к папке с CSV файлами связки вакансий и дисциплин
path_to_files_u = Path("/Users/dl/Library/Mobile Documents/com~apple~CloudDocs/Работа/ВШЭ/RecSys_24-25/Связка_вакансии_дисциплины")

# Определение новой папки для сохранения обновленных файлов
output_dir = Path("/Users/dl/Library/Mobile Documents/com~apple~CloudDocs/Работа/ВШЭ/RecSys_24-25/Связка_вакансии_дисциплины_2")

# Создание новой папки или очистка существующей
if output_dir.exists():
    shutil.rmtree(output_dir)  # Удаляем существующую папку со всем содержимым
output_dir.mkdir(parents=True)  # Создаем новую папку

# Получение списка всех CSV файлов
csv_files_u = list(path_to_files_u.rglob("*.csv"))

# Обработка файлов связки вакансий и дисциплин
for file_path, file_path_u in tqdm(zip(csv_files, csv_files_u), desc="Обработка файлов", total=len(csv_files), unit="файл"):
    # Чтение первых 10 строк файла вакансий
    df = pd.read_csv(file_path, sep=';', nrows=10)

    # Чтение файла связки
    df_u = pd.read_csv(file_path_u, sep=';')

    # Проверка, что файл связки содержит только 10 строк
    if len(df_u) != 10:
        print(f"Файл {file_path_u.name} не содержит 10 строк, он будет пропущен.")
        continue

    # Обновление ID в файле связки на первые 10 ID из файла вакансий
    df_u['ID'] = df['ID'].values

    # Перемещение столбца 'ID' на первое место
    cols = ['ID'] + [col for col in df_u.columns if col != 'ID']
    df_u = df_u[cols]

    # Определение пути для сохранения обновленного файла
    new_file_path = output_dir / file_path_u.name

    # Сохранение обновленного файла
    df_u.to_csv(new_file_path, index=False, encoding="utf-8-sig", sep=";")

print("Обработка завершена.")

Обработка файлов: 100%|████████████████████████████████████████████████████████████████████████| 117/117 [00:02<00:00, 46.60файл/s]

Обработка завершена.


In [7]:
path_to_files = "/Users/dl/Library/Mobile Documents/com~apple~CloudDocs/Работа/ВШЭ/RecSys_24-25/ПУДы"

xlsx_files = glob.glob(os.path.join(path_to_files, "*.xlsx"))

df = pd.DataFrame()

for file in xlsx_files:
    df_temp = pd.read_excel(file)
    df = pd.concat([df, df_temp], ignore_index = True)

In [12]:
df_u.iloc[0]

ID                                                                        103566666
Name                                Руководитель группы по организации производства
Description                       Обязанности:  Выполнять устные и письменные пр...
KeySkills                                                                       NaN
ProfessionalRoles                                     Руководитель отдела логистики
City                                                                         Тамбов
SBERT_plus_LLM_Recommendations    Лидерство и построение команды в бизнесе; Упра...
SBERT_Disciplines                 Лидерство, коммуникации и персональная эффекти...
Name: 0, dtype: object

In [31]:
import os
import shutil
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import re

# Функция для обработки каждой строки
def process_row(row, df, column_name):
    if pd.isna(row[column_name]):
        return row[column_name]

    # Разделяем строку по ';'
    courses = [course.strip() for course in row[column_name].split(';')]

    processed_courses = []
    for course in courses:
        escaped_course = re.escape(course)
        
        # Находим совпадение в "Русскоязычное название дисциплины"
        match = df[df['Русскоязычное название дисциплины'].str.contains(escaped_course, na=False, case=False)]

        if not match.empty:
            # Сортируем совпадения по периоду изучения дисциплины (по убыванию)
            match['year'] = match['Период изучения дисциплины'].str.extract(r'(\d{4}/\d{4})').fillna('0000/0000')
            match = match.sort_values(by='year', ascending=False)
            first_match = match.iloc[0]

            # Формируем новую строку с нужными данными и заменяем nan на прочерк
            campus = first_match['Кампус кафедры, предлагающей дисциплину'] if not pd.isna(first_match['Кампус кафедры, предлагающей дисциплину']) else '-'
            faculty = first_match['Факультет кафедры, предлагающей дисциплину'] if not pd.isna(first_match['Факультет кафедры, предлагающей дисциплину']) else '-'
            department = first_match['Кафедра, предлагающая дисциплину'] if not pd.isna(first_match['Кафедра, предлагающая дисциплину']) else '-'
            level = first_match['Уровень обучения'] if not pd.isna(first_match['Уровень обучения']) else '-'
            period = first_match['Период изучения дисциплины'] if not pd.isna(first_match['Период изучения дисциплины']) else '-'
            
            new_course = f"{course} | {campus} | {faculty} | {department} | {level} | {period}"

            processed_courses.append(new_course)
        else:
            # Если совпадений нет, добавляем исходное название курса
            processed_courses.append(course)

    # Склеиваем обработанные курсы обратно в одну строку
    return '; '.join(processed_courses)

# Путь к исходной и целевой директориям
path_to_files = Path("/Users/dl/Downloads/results-2_")
save_path = Path("/Users/dl/Downloads/results-3_")

# Проверка и создание/очистка целевой папки
if save_path.exists():
    shutil.rmtree(save_path)  # Удаляем все файлы и папки внутри
save_path.mkdir(parents=True, exist_ok=True)  # Создаем целевую папку

# Получение списка всех CSV файлов
csv_files = list(path_to_files.rglob("*.csv"))

# Проходим по каждому файлу
for file_path in tqdm(csv_files, desc="Обработка CSV файлов", unit="файл"):
    # Чтение CSV файла
    df_u = pd.read_csv(file_path, sep=';')
    
    # Применяем функцию к столбцам SBERT_plus_LLM_Recommendations и SBERT_Disciplines
    df_u['SBERT_plus_LLM_Recommendations'] = df_u.apply(lambda row: process_row(row, df, 'SBERT_plus_LLM_Recommendations'), axis=1)
    df_u['SBERT_Disciplines'] = df_u.apply(lambda row: process_row(row, df, 'SBERT_Disciplines'), axis=1)
    
    # Определение пути для сохранения файла
    save_file_path = save_path / file_path.name
    
    # Сохранение обработанного DataFrame в CSV
    df_u.to_csv(save_file_path, index=False, encoding="utf-8-sig", sep=";")

print("Обработка и сохранение завершены.")


Обработка CSV файлов: 100%|████████████████████████████████████████████████████████████████████| 117/117 [45:01<00:00, 23.09s/файл]

Обработка и сохранение завершены.
